In [ ]:
from pandas import ExcelWriter
from pandas import ExcelFile

In [ ]:
from matplotlib.pylab import rcParams
import xlrd
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping


In [ ]:
from datetime import datetime

In [ ]:
rcParams["figure.figsize"] = 20,20

def parser(x):
    return datetime.strptime(x, "%m/%d/%y")

df = pd.read_excel(
     os.path.join("test.xlsx"),sheet_name="Data", engine='openpyxl', parse_dates=[0], date_parser = parser
)


In [ ]:
df

In [ ]:
df.drop(["Severity",'Case ID', "Status","Total" ],axis = 1, inplace =True)

In [ ]:
df = df.groupby(["Entered on"], as_index = True).mean()
df = df.rename(columns={'Resolution Time': 'value'})
df = df.fillna(0)
df.plot()

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot
def plot_dataset(df, title):
    data = []
    value = go.Scatter(
        x=df.index,
        y=df.iloc[:,0],
        mode="lines",
        name="incidents",
        marker=dict(),
        text=df.index,
        line=dict(color="rgba(1,0,1, 0.3)"),
        
        
        
    )
    data.append(value)
        

    layout = dict(
        title=title,
        xaxis=dict(title="Date", ticklen=5, zeroline=False),
        yaxis=dict(title="Incidents", ticklen=5, zeroline=False),
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)

In [ ]:
plot_dataset(df, "Number of Incidents")

In [ ]:
from matplotlib.pylab import rcParams
%matplotlib inline
import xlrd
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

In [ ]:
from sklearn.model_selection import train_test_split

close_data = df['value'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

X_train = close_data[:split]
y_test = close_data[split:]


print(len(X_train))
print(len(close_test))




In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler

def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
scaler = get_scaler('minmax')

scaler = scaler.fit(X_train)

train_data = scaler.transform(X_train)

test_data = scaler.transform(y_test)


In [ ]:
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

In [ ]:
seq_length = 5
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        """The __init__ method that initiates an RNN instance.

        Args:
            input_dim (int): The number of nodes in the input layer
            hidden_dim (int): The number of nodes in each layer
            layer_dim (int): The number of layers in the network
            output_dim (int): The number of nodes in the output layer
            dropout_prob (float): The probability of nodes being dropped out

        """
        super(RNNModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """The forward method takes input tensor x and does forward propagation

        Args:
            x (torch.Tensor): The input tensor of the shape (batch size, sequence length, input_dim)

        Returns:
            torch.Tensor: The output tensor of the shape (batch size, output_dim)

        """
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, h0 = self.rnn(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out

In [ ]:
class LSTMModel(nn.Module):

  def __init__(self, n_features, n_hidden, seq_len, n_layers=2):
    super(LSTMModel, self).__init__()

    self.n_hidden = n_hidden
    self.seq_len = seq_len
    self.n_layers = n_layers

    self.lstm = nn.LSTM(
      input_size=n_features,
      hidden_size=n_hidden,
      num_layers=n_layers,
      dropout=0.5
    )

    self.linear = nn.Linear(in_features=n_hidden, out_features=1)

  def reset_hidden_state(self):
    self.hidden = (
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
    )

  def forward(self, sequences):
    lstm_out, self.hidden = self.lstm(
      sequences.view(len(sequences), self.seq_len, -1),
      self.hidden
    )
    last_time_step = \
      lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
    y_pred = self.linear(last_time_step)
    return y_pred

In [ ]:
class GRUModel(nn.Module):
    """GRUModel class extends nn.Module class and works as a constructor for GRUs.

       GRUModel class initiates a GRU module based on PyTorch's nn.Module class.
       It has only two methods, namely init() and forward(). While the init()
       method initiates the model with the given input parameters, the forward()
       method defines how the forward propagation needs to be calculated.
       Since PyTorch automatically defines back propagation, there is no need
       to define back propagation method.

       Attributes:
           hidden_dim (int): The number of nodes in each layer
           layer_dim (str): The number of layers in the network
           gru (nn.GRU): The GRU model constructed with the input parameters.
           fc (nn.Linear): The fully connected layer to convert the final state
                           of GRUs to our desired output shape.

    """
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        """The __init__ method that initiates a GRU instance.

        Args:
            input_dim (int): The number of nodes in the input layer
            hidden_dim (int): The number of nodes in each layer
            layer_dim (int): The number of layers in the network
            output_dim (int): The number of nodes in the output layer
            dropout_prob (float): The probability of nodes being dropped out

        """
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """The forward method takes input tensor x and does forward propagation

        Args:
            x (torch.Tensor): The input tensor of the shape (batch size, sequence length, input_dim)

        Returns:
            torch.Tensor: The output tensor of the shape (batch size, output_dim)

        """
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [ ]:
def get_model(model, model_params):
    models = {
        "rnn": RNNModel,
        "lstm": LSTMModel,
        "gru": GRUModel,
    }
    return models.get(model.lower())(**model_params)

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt

   

In [2]:
def train_model(
  model,
  train_data,
  train_labels,
  test_data=None,
  test_labels=None
):
        
          loss_fn = torch.nn.MSELoss(reduction='sum')
          optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)
          num_epochs = 60
          train_hist = np.zeros(num_epochs)
          test_hist = np.zeros(num_epochs)
          for t in range(num_epochs):
            model.reset_hidden_state()
            y_pred = model(X_train)
            loss = loss_fn(y_pred.float(), y_train)
            if test_data is not None:
              with torch.no_grad():
                y_test_pred = model(X_test)
                test_loss = loss_fn(y_test_pred.float(), y_test)
              test_hist[t] = test_loss.item()
              if t % 10 == 0:
                print(f'Epoch {t} train loss: {loss.item()} test loss: {test_loss.item()}')
            elif t % 10 == 0:
              print(f'Epoch {t} train loss: {loss.item()}')
            train_hist[t] = loss.item()
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
          return model.eval(), train_hist, test_hist

def plot_losses(train_hist,test_his):
    plt.plot(train_hist, label="Training loss")
    plt.plot(test_hist, label="Test loss")
    plt.ylim((0, 5))
    plt.legend();
        
plot_losses(train_hist,test_his)  

In [ ]:
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
model = LSTMModel(
  n_features=1, 
  n_hidden=64, 
  seq_len=seq_length, 
  n_layers=2
)
model, train_hist, test_hist = train_model(
  model, 
  X_train, 
  y_train, 
  X_test, 
  y_test
)

In [3]:
#predicting daily cases
with torch.no_grad():
  test_seq = X_test[:1]
  preds = []
  for _ in range(len(X_test)):
    y_test_pred = model(test_seq)
    pred = torch.flatten(y_test_pred).item()
    preds.append(pred)
    new_seq = test_seq.numpy().flatten()
    new_seq = np.append(new_seq, [pred])
    new_seq = new_seq[1:]
    test_seq = torch.as_tensor(new_seq).view(1, seq_length, 1).float()

NameError: name 'torch' is not defined

In [ ]:
#inverse the data
true_cases = scaler.inverse_transform(
    np.expand_dims(y_test.flatten().numpy(), axis=0)
).flatten()

predicted_cases = scaler.inverse_transform(
  np.expand_dims(preds, axis=0)
).flatten()

In [ ]:
plt.plot(
  daily_cases.index[:len(train_data)], 
  scaler.inverse_transform(train_data).flatten(),
  label='Historical Daily Cases'
)

plt.plot(
  daily_cases.index[len(train_data):len(train_data) + len(true_cases)], 
  true_cases,
  label='Real Daily Cases'
)

plt.plot(
  daily_cases.index[len(train_data):len(train_data) + len(true_cases)], 
  predicted_cases, 
  label='Predicted Daily Cases'
)

plt.legend();

In [4]:

scaler = MinMaxScaler()

scaler = scaler.fit(df)

all_data = scaler.transform(df)

all_data.shape

NameError: name 'MinMaxScaler' is not defined

In [ ]:
X_all, y_all = create_sequences(all_data, seq_length)

X_all = torch.from_numpy(X_all).float()
y_all = torch.from_numpy(y_all).float()

model = CoronaVirusPredictor(
  n_features=1, 
  n_hidden=512, 
  seq_len=seq_length, 
  n_layers=2
)
model, train_hist, _ = train_model(model, X_all, y_all)

In [ ]:
DAYS_TO_PREDICT = 12

with torch.no_grad():
  test_seq = X_all[:1]
  preds = []
  for _ in range(DAYS_TO_PREDICT):
    y_test_pred = model(test_seq)
    pred = torch.flatten(y_test_pred).item()
    preds.append(pred)
    new_seq = test_seq.numpy().flatten()
    new_seq = np.append(new_seq, [pred])
    new_seq = new_seq[1:]
    test_seq = torch.as_tensor(new_seq).view(1, seq_length, 1).float()

In [ ]:
predicted_cases = scaler.inverse_transform(
  np.expand_dims(preds, axis=0)
).flatten()

In [ ]:
predicted_index = pd.date_range(
  start=daily_cases.index[-1],
  periods=DAYS_TO_PREDICT + 1,
  closed='right'
)

predicted_cases = pd.Series(
  data=predicted_cases,
  index=predicted_index
)

plt.plot(predicted_cases, label='Predicted Daily Cases')
plt.legend();

In [ ]:
plt.plot(daily_cases, label='Historical Daily Cases')
plt.plot(predicted_cases, label='Predicted Daily Cases')
plt.legend();

In [ ]:


input_dim = len(X_train.columns)
output_dim = 1
hidden_dim = 32
layer_dim = 2
batch_size = 32
dropout = 0.2
n_epochs = 50
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {'input_dim': input_dim,
                'hidden_dim' : hidden_dim,
                'layer_dim' : layer_dim,
                'output_dim' : output_dim,
                'dropout_prob' : dropout}

model = get_model('lstm', model_params)

loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
opt.plot_losses()

predictions, values = opt.evaluate(
    test_loader_one,
    batch_size=1,
    n_features=input_dim
)
test_loader_iter = iter(test_loader)
*_, (X, y) = test_loader_iter
forecast = forecast(X, y, batch_size=batch_size, n_features=input_dim, n_steps=2)

In [ ]:
def inverse_transform(scaler, df, columns):
    for col in columns:
        df[col] = scaler.inverse_transform(df[col])
    return df


def format_predictions(predictions, values, df_test, scaler):
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate(predictions, axis=0).ravel()
    df_result = pd.DataFrame(data={"value": vals, "prediction": preds}, index=df_test.head(len(vals)).index)
    df_result = df_result.sort_index()
    df_result = inverse_transform(scaler, df_result, [["value", "prediction"]])
    return df_result


df_result = format_predictions(predictions, values, X_test, scaler)
df_result
# df_result.to_excel('Tablea_Motion_Chart.xlsx', sheet_name='new_sheet_name')

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot


def plot_predictions(df_result, df_baseline):
    data = []
    
    value = go.Scatter(
        x=df_result.index,
        y=df_result.value,
        mode="lines",
        name="values",
        marker=dict(),
        text=df_result.index,
        line=dict(color="rgba(0,0,0, 0.3)"),
    )
    data.append(value)

#     baseline = go.Scatter(
#         x=df_baseline.index,
#         y=df_baseline.prediction,
#         mode="lines",
#         line={"dash": "dot"},
#         name='linear regression',
#         marker=dict(),
#         text=df_baseline.index,
#         opacity=0.8,
#     )
#     data.append(baseline)
    
    prediction = go.Scatter(
        x=df_result.index,
        y=df_result.prediction,
        mode="lines",
        line={"dash": "dot"},
        name='predictions',
        marker=dict(),
        text=df_result.index,
        opacity=0.8,
    )
    data.append(prediction)
    
    layout = dict(
        title="Predictions vs Actual Values for the dataset",
        xaxis=dict(title="Time", ticklen=5, zeroline=False),
        yaxis=dict(title="Value", ticklen=5, zeroline=False),
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)
    
    
# Set notebook mode to work in offline
pyo.init_notebook_mode()

plot_predictions(df_result, df_features)

In [ ]:
def forecast(self, X, y, batch_size=1, n_features=1, n_steps=100):
        predictions1 = []
        
        test_loader_iter = iter(test_loader)
        *_, (X, y) = test_loader_iter
        y = y.to(device).detach().numpy()
        X = X.view([batch_size, -1, n_features]).to(device)
        X = torch.roll(X, shifts=1, dims=2)
        X[..., -1, 0] = y.item(0)
             
        with torch.no_grad():
            self.model.eval()
            for _ in range(n_steps):
                X = X.view([batch_size, -1, n_features]).to(device)
                yhat = self.model(X)
                yhat = yhat.to(device).detach().numpy()
                X = torch.roll(X, shifts=1, dims=2)
                X[..., -1, 0] = yhat.item(0)
                predictions1.append(yhat.item(0))

        return predictions1

test_loader_iter = iter(test_loader)
*_, X, y = test_loader_iter
forecast(X, y,  batch_size=batch_size, n_features=input_dim, n_steps=2)

In [ ]:
num_prediction = 20
#start the predcition from last time point +1
prediction_dates = pd.date_range("2021-07-31", periods=num_prediction, freq="D").tolist()
series_column = pd.DataFrame(prediction_dates, columns= ["Entered on"])
series = series_column.set_index("Entered on")

In [ ]:
series_features = (
                series       
                .assign(month = series.index.month)
                .assign(day_of_week = series.index.dayofweek)
                .assign(week_of_year = series.index.week))
series_features = onehot_encode_pd(series_features, ["month","day_of_week",'week_of_year'])


In [ ]:
def generate_cyclical_features(df, col_name, period, start_num=0):
    kwargs = {
        f'sin_{col_name}' : lambda x: np.sin(2*np.pi*(df[col_name]-start_num)/period),
        f'cos_{col_name}' : lambda x: np.cos(2*np.pi*(df[col_name]-start_num)/period)    
             }
    return df.assign(**kwargs).drop(columns=[col_name])

# df_features = generate_cyclical_features(df_features, 'hour', 24, 0)
# df_features = generate_cyclical_features(df_features, 'day_of_week', 7, 0)
# df_features = generate_cyclical_features(df_features, 'month', 12, 1)
# series_features = generate_cyclical_features(series_features, 'week_of_year', 52, 0)

In [ ]:
def feature_label_split(df, index):
    y = series_column[["Entered on"]]
    X = df
    return X, y
X, y = feature_label_split(series_features, series.index)
scaler = get_scaler('minmax')
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)


batch_size = 1

test_features = torch.Tensor(X)
test_targets = torch.Tensor(y)


test = TensorDataset(test_targets, test_features)

test_loader_forecast = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)


In [ ]:
len(series_features.columns)

In [ ]:
# test_loader_iter = iter(test_loader_forecast)
# *_, (X, y) = test_loader_iter

predictions2 = forecast_features(test_loader_features, 
                                 batch_size=1, 
                                 n_features=len(series_features.columns)
                                )



In [ ]:
#transform into readbale format
vals = np.concatenate(values, axis=0).ravel()
preds = np.concatenate(predictions, axis=0).ravel()
#make a DF
df_result = pd.DataFrame(data={"value": vals, "prediction": preds}, index=X_test.head(len(vals)).index)
df_result = df_result.sort_index()
# df_result = inverse_transform(scaler, df_result, [["value", "prediction"]])

#create DF for new time
num_prediction = 30
#start the predcition from last time point +1
prediction_dates = pd.date_range("2021-07-30", periods=num_prediction, freq="D").tolist()
series = pd.DataFrame(prediction_dates, columns= ["Entered on"])
# put the new tile line with predictions
preds = np.stack(predictions1)
# preds = np.concatenate(predictions1, axis=0).ravel()

forecast1 = pd.DataFrame(preds, columns=['future'])
df = pd.concat([series, forecast1], axis=1)
df = df.set_index(df["Entered on"])
future = df.drop(columns = ["Entered on"])

   

#concat both values
result = pd.concat([df_result["value"], df["future"], df_result["prediction"]], axis=1)
result = inverse_transform(scaler, result, [["value", "future", "prediction"]])


In [ ]:
preds

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot


def plot_predictions(df_result, df_baseline):
    data = []
    
    value = go.Scatter(
        x= result.index,
        y= result.value,
        mode="lines",
        name="values",
        marker=dict(),
        text=df_result.index,
        line=dict(color="rgba(0,0,0, 0.3)"),
    )
    data.append(value)
    
    future= go.Scatter(
        x= result.index,
        y= result.future,
        mode="lines",
        name="future",
        marker=dict(),
        text=df_result.index,
        
    )
    data.append(future)


    
    prediction = go.Scatter(
        x= result.index,
        y= result.prediction,
        mode="lines",
        line={"dash": "dot"},
        name='predictions',
        marker=dict(),
        text=df_result.index,
        opacity=0.8,
    )
    data.append(prediction)
    
    layout = dict(
        title="Forecast the future",
        xaxis=dict(title="Time", ticklen=5, zeroline=False),
        yaxis=dict(title="Value", ticklen=5, zeroline=False),
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)
    
    
# Set notebook mode to work in offline
pyo.init_notebook_mode()

plot_predictions(df_result, df_features)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(df):
    result_metrics = {'mae' : mean_absolute_error(df.value, df.prediction),
                      'rmse' : mean_squared_error(df.value, df.prediction) ** 0.5,
                      'r2' : r2_score(df.value, df.prediction)}
    
    print("Mean Absolute Error:       ", result_metrics["mae"])
    print("Root Mean Squared Error:   ", result_metrics["rmse"])
    print("R^2 Score:                 ", result_metrics["r2"])
    return result_metrics

result_metrics = calculate_metrics(df_result)

In [ ]:
# from sklearn.linear_model import LinearRegression

# def build_baseline_model(df, test_ratio, target_col):
#     X, y = feature_label_split(df, target_col)
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_ratio, shuffle=False
#     )
#     model = LinearRegression()
#     model.fit(X_train, y_train)
#     prediction = model.predict(X_test)

#     result = pd.DataFrame(y_test)
#     result["prediction"] = prediction
#     result = result.sort_index()

#     return result

# df_baseline = build_baseline_model(df_features, 0.2, 'value')
# baseline_metrics = calculate_metrics(df_baseline)

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot


def plot_predictions(df_result, df_baseline):
    data = []
    
    value = go.Scatter(
        x=df_result.index,
        y=df_result.value,
        mode="lines",
        name="values",
        marker=dict(),
        text=df_result.index,
        line=dict(color="rgba(0,0,0, 0.3)"),
    )
    data.append(value)

#     baseline = go.Scatter(
#         x=df_baseline.index,
#         y=df_baseline.prediction,
#         mode="lines",
#         line={"dash": "dot"},
#         name='linear regression',
#         marker=dict(),
#         text=df_baseline.index,
#         opacity=0.8,
#     )
#     data.append(baseline)
    
    prediction = go.Scatter(
        x=df_result.index,
        y=df_result.prediction,
        mode="lines",
        line={"dash": "dot"},
        name='predictions',
        marker=dict(),
        text=df_result.index,
        opacity=0.8,
    )
    data.append(prediction)
    
    layout = dict(
        title="Predictions vs Actual Values for the dataset",
        xaxis=dict(title="Time", ticklen=5, zeroline=False),
        yaxis=dict(title="Value", ticklen=5, zeroline=False),
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)
    
    
# Set notebook mode to work in offline
pyo.init_notebook_mode()

plot_predictions(df_result, df_features)

In [ ]:
def plot_predictions(df_result, df_baseline):
    data = []
    
    value = go.Scatter(
        x=df_result.index,
        y=df_result.value,
        mode='lines',
        line= dict(color="rgba(1,0,1, 0.3)",
                  width=1.5))
       
        
        
    
    data.append(value)

    prediction = go.Scatter(
        x=df_result.index,
        y=df_result.prediction,
        mode="lines",
        line=dict(width=1.5, dash="dashdot"))
    
    
    data.append(prediction)
    
    frames = [dict(data= [dict(type='scatter',
                           x=df_result.index, 
                               y=df_result.value),
                          dict(type='scatter',
                           x=df_result.index, 
                               y=df_result.prediction[:k+1])],
               traces= [0,1],  
              )for k  in  range(1, len(df_result)-1)]
    layout = go.Layout(title="Predictions vs Actual Values for the dataset",
        xaxis=dict(title="Time", ticklen=5, zeroline=False),
        yaxis=dict(title="Value", ticklen=5, zeroline=False),
                       width=900,
                   height=600,
                   showlegend=False,
                   hovermode='x unified',
                   updatemenus=[
                        dict(
                            type='buttons', showactive=False,
                            y=1.05,
                            x=1.15,
                            xanchor='right',
                            yanchor='top',
                            pad=dict(t=0, r=10),
                            buttons=[dict(label='Play',
                            method='animate',
                            args=[None, 
                                  dict(frame=dict(duration=0, 
                                                  redraw=False),
    transition=dict(duration=0),
                                                  fromcurrent=True,
                                                  mode='immediate')]
                            )]
                        ),
                        
                    ]              
                  )
    layout.update(xaxis =dict(range=[df_result.index[0], df_result.index[-1]], autorange=False),
                  yaxis =dict(range=[0, max(df_result.prediction)], autorange=False));
    fig = go.Figure(data=[value, prediction], frames=frames, layout=layout)
    fig.show()
    import plotly.express as px
    fig.write_html("Animation.html")

    
pyo.init_notebook_mode()
plot_predictions(df_result, df_features)